In [1]:
import pandas as pd
import joblib

In [6]:
ratings = pd.read_csv('ratings.csv')
ratings = ratings.drop(['timestamp'], axis=1)
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [7]:
user_ratings = ratings.pivot_table(index=['userId'],columns=['movieId'],values='rating')
user_ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
## Let's Remove Movies which have less than 10 users who rated it. and fill remaining NaN with 0
user_ratings = user_ratings.dropna(thresh=10,axis=1).fillna(0)
user_ratings.head()

movieId,1,2,3,5,6,7,9,10,11,12,...,166461,166528,166643,168250,168252,174055,176371,177765,179819,187593
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
## Let's Build our Similarity Matrix
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50)  

movieId,1,2,3,5,6,7,9,10,11,12,...,166461,166528,166643,168250,168252,174055,176371,177765,179819,187593
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.231327,0.173213,0.192474,0.192686,0.143743,0.177245,0.183382,0.172799,0.159352,...,-0.003950,0.070499,0.111588,0.022451,0.066439,0.059796,0.054642,-0.013607,0.053028,0.022160
2,0.231327,1.000000,0.191945,0.200526,0.158341,0.127569,-0.021045,0.285086,0.217090,0.115290,...,0.047793,0.167779,0.049701,0.036534,0.195111,0.060834,0.150281,-0.020637,0.113709,0.118390
3,0.173213,0.191945,1.000000,0.370171,0.196442,0.351513,0.275812,0.136916,0.174251,0.168038,...,-0.035086,0.000094,0.018855,-0.031249,-0.000959,0.044769,0.033825,-0.039794,0.026881,-0.039206
5,0.192474,0.200526,0.370171,1.000000,0.215503,0.429890,0.308085,0.110833,0.201002,0.173630,...,0.029139,0.006434,-0.005243,-0.004671,-0.022762,-0.003536,0.000280,-0.038865,-0.000983,-0.038291
6,0.192686,0.158341,0.196442,0.215503,1.000000,0.148109,0.167909,0.251343,0.182082,0.115893,...,-0.052982,0.106208,0.006448,0.038345,0.123248,0.078585,0.119709,-0.060090,0.015800,0.038239
7,0.143743,0.127569,0.351513,0.429890,0.148109,1.000000,0.124458,0.129010,0.312137,0.130808,...,-0.035995,-0.002884,0.004503,-0.023351,-0.002858,0.041336,0.029589,-0.040824,0.010491,-0.040222
9,0.177245,-0.021045,0.275812,0.308085,0.167909,0.124458,1.000000,-0.024328,-0.013803,0.328872,...,-0.019023,-0.015814,-0.019839,0.000177,-0.012876,0.014197,-0.001716,-0.021576,-0.020035,-0.021257
10,0.183382,0.285086,0.136916,0.110833,0.251343,0.129010,-0.024328,1.000000,0.235899,0.091678,...,-0.023759,0.039676,-0.037301,-0.029683,0.024680,-0.009848,0.032775,-0.031616,-0.028194,0.014762
11,0.172799,0.217090,0.174251,0.201002,0.182082,0.312137,-0.013803,0.235899,1.000000,-0.044891,...,-0.042251,0.005043,0.010982,-0.012735,0.002639,0.042380,0.041955,-0.035557,0.041405,-0.013474


In [11]:
joblib.dump(item_similarity_df,"item_similarity_df")

['item_similarity_df']

In [13]:
def get_similar_movies(movieId,user_rating):
    similar_score = item_similarity_df[movieId]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [15]:
get_similar_movies(10,4)

movieId
10        1.500000
165       0.768348
349       0.713569
380       0.670547
316       0.669511
            ...   
106100   -0.118557
104879   -0.121703
109487   -0.127025
115569   -0.131288
92259    -0.143097
Name: 10, Length: 2269, dtype: float64